In [78]:
import glob
import pandas as pd

# Wczytanie danych

In [96]:
csv_fnames = glob.glob('Sales*')
sales_df = pd.DataFrame()
for csv_fname in csv_fnames:
    sales_df = pd.concat([sales_df, pd.read_csv(csv_fname).dropna()], ignore_index=True)
sales_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186300,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186301,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186302,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186303,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


# Czyszczenie danych

In [97]:
sales_df.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

In [98]:
sales_df['Order ID'] = pd.to_numeric(sales_df['Order ID'], errors='coerce')
sales_df = sales_df.dropna().astype({'Order ID':'int32'})

In [99]:
sales_df['Quantity Ordered'] = pd.to_numeric(sales_df['Quantity Ordered'], errors='coerce')
sales_df = sales_df.dropna().astype({'Quantity Ordered':'int16'})

In [100]:
sales_df['Price Each'] = pd.to_numeric(sales_df['Price Each'], errors='coerce')
sales_df = sales_df.dropna()

In [101]:
sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'], errors='coerce')
sales_df = sales_df.dropna()

In [102]:
sales_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186300,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001"
186301,259354,iPhone,1,700.00,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016"
186302,259355,iPhone,1,700.00,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016"
186303,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016"


In [103]:
sales_df.dtypes

Order ID                     int32
Product                     object
Quantity Ordered             int16
Price Each                 float64
Order Date          datetime64[ns]
Purchase Address            object
dtype: object

# Zadanie 1

Przygotować zestawienie przedstawiające całkowite sumy kwot sprzedaży dla każdego zapisanego dnia w 2019 roku.

In [104]:
sales_df['Price Full'] = sales_df['Quantity Ordered'] * sales_df['Price Each']

In [105]:
price_sum_by_day_df = pd.DataFrame(sales_df.groupby(pd.Grouper(key='Order Date', axis=0, freq='D', sort=True)).sum()['Price Full'])
price_sum_by_day_df

,Price Full
Order Date,
2019-01-01,65681.94
2019-01-02,70813.20
2019-01-03,47046.20
2019-01-04,62012.21
2019-01-05,46524.63
...,...
2019-12-28,134015.50
2019-12-29,156024.62
2019-12-30,152319.81


# Zadanie 2

Przygotować zestawienie przedstawiające 3 produkty, których sprzedano najwięcej oraz ich łączną sumę kwot sprzedaży i liczbę sprzedanych sztuk dla każdego kwartału 2019 roku.

In [142]:
sales_by_quarter_df = sales_df.groupby([pd.Grouper(key='Order Date', axis=0, freq='4M', sort=True), 'Product']).sum()[['Price Full', 'Quantity Ordered']]
sales_by_quarter_df

Price Full  Quantity Ordered
Order Date Product                                             
2019-01-31 20in Monitor              23647.85               215
           27in 4K Gaming Monitor   121676.88               312
           27in FHD Monitor          62845.81               419
           34in Ultrawide Monitor   119316.86               314
           AA Batteries (4-pack)      5468.16              1424
...                                       ...               ...
2020-01-31 ThinkPad Laptop         1375986.24              1376
           USB-C Charging Cable      97715.15              8177
           Vareebadd Phone          266000.00               665
           Wired Headphones          83462.39              6961
           iPhone                  1602300.00              2289

[76 rows x 2 columns]

In [143]:
quarter_indexes = sales_by_quarter_df.index.get_level_values(0).unique()
quarter_indexes

DatetimeIndex(['2019-01-31', '2019-05-31', '2019-09-30', '2020-01-31'], dtype='datetime64[ns]', name='Order Date', freq=None)

In [155]:
top_3_sales_by_quarter_df = pd.DataFrame()

In [156]:
for idx in quarter_indexes:
    temp_df = pd.DataFrame(sales_by_quarter_df.loc[idx].nlargest(3, 'Quantity Ordered'))
    temp_df['Quarter'] = str(idx)
    top_3_sales_by_quarter_df = pd.concat([top_3_sales_by_quarter_df, temp_df])

In [161]:
top_3_sales_by_quarter_df.reset_index().set_index(['Quarter','Product'])

Price Full  Quantity Ordered
Quarter             Product                                             
2019-01-31 00:00:00 AAA Batteries (4-pack)     4769.05              1595
                    AA Batteries (4-pack)      5468.16              1424
                    USB-C Charging Cable      15331.85              1283
2019-05-31 00:00:00 AAA Batteries (4-pack)    30847.83             10317
                    AA Batteries (4-pack)     35159.04              9156
                    USB-C Charging Cable      94823.25              7935
2019-09-30 00:00:00 AAA Batteries (4-pack)    25328.29              8471
                    AA Batteries (4-pack)     29821.44              7766
                    USB-C Charging Cable      78631.00              6580
2020-01-31 00:00:00 AAA Batteries (4-pack)    31795.66             10634
                    AA Batteries (4-pack)     35669.76              9289
                    USB-C Charging Cable      97715.15              8177

# Zadanie 3

Przygotować zestawienie przedstawiające 5 stanów, które najczęściej powtarzają się w adresach zamówień oraz ich udział w liczbie wszystkich zamówień w 2019 roku.

In [175]:
sales_df['State'] = sales_df['Purchase Address'].apply(lambda x: x.split(',')[1].strip())
sales_df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Price Full,State
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",23.90,Dallas
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",99.99,Boston
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",600.00,Los Angeles
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",11.99,Los Angeles
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",11.99,Los Angeles
...,...,...,...,...,...,...,...,...
186300,259353,AAA Batteries (4-pack),3,2.99,2019-09-17 20:56:00,"840 Highland St, Los Angeles, CA 90001",8.97,Los Angeles
186301,259354,iPhone,1,700.00,2019-09-01 16:00:00,"216 Dogwood St, San Francisco, CA 94016",700.00,San Francisco
186302,259355,iPhone,1,700.00,2019-09-23 07:39:00,"220 12th St, San Francisco, CA 94016",700.00,San Francisco
186303,259356,34in Ultrawide Monitor,1,379.99,2019-09-19 17:30:00,"511 Forest St, San Francisco, CA 94016",379.99,San Francisco


In [196]:
quantiy_of_orders = len(sales_df['Order ID'].unique())
quantiy_of_orders

178437

In [200]:
pd.DataFrame(sales_df.groupby(['State', 'Order ID']).size().groupby('State').size().nlargest(5)/quantiy_of_orders, columns=['StateShare'])

,StateShare
State,
San Francisco,0.240410
Los Angeles,0.159709
New York City,0.133649
Boston,0.106996
Atlanta,0.079877
